In [17]:
import polars as pl
import glob
import os
from tqdm.autonotebook import tqdm
ids=[]
points=[]
size=[]
for f in tqdm(glob.glob("/home/liontao/work/data/geolife/Data/**/Trajectory/*.plt")):
    ids.append(os.path.splitext(os.path.split(f)[-1])[0])
    single_traj = pl.read_csv(f, has_header=False, skip_rows=6, new_columns=["lat","lon","zero","alt","days","date","time"], dtypes=[pl.datatypes.Float64, pl.datatypes.Float64, pl.datatypes.Int32, pl.datatypes.Float64, pl.datatypes.Float64, pl.datatypes.Datetime,pl.datatypes.Datetime])
    points.append(";".join([",".join(map(str,i)) for i in zip(single_traj["lon"],(single_traj["lat"]))]))
    size.append(len(single_traj["lon"]))
geolife_data = pl.DataFrame({"id":ids,"points":points,"size":size})
print(geolife_data)

  0%|          | 0/18670 [00:00<?, ?it/s]

shape: (18_670, 3)
┌────────────────┬───────────────────────────────────┬──────┐
│ id             ┆ points                            ┆ size │
│ ---            ┆ ---                               ┆ ---  │
│ str            ┆ str                               ┆ i64  │
╞════════════════╪═══════════════════════════════════╪══════╡
│ 20081023025304 ┆ 116.318417,39.984702;116.31845,3… ┆ 908  │
│ 20081024020959 ┆ 116.319876,40.008304;116.319962,… ┆ 244  │
│ 20081026134407 ┆ 116.370017,39.907414;116.370074,… ┆ 745  │
│ 20081027115449 ┆ 116.326757,39.994622;116.326757,… ┆ 50   │
│ …              ┆ …                                 ┆ …    │
│ 20080206025637 ┆ 112.965233333333,28.16815;112.97… ┆ 49   │
│ 20080216080442 ┆ 116.30895,39.9854666666667;116.3… ┆ 33   │
│ 20080217010131 ┆ 116.302783333333,39.989933333333… ┆ 61   │
│ 20080314025755 ┆ 111.709216666667,40.91815;111.70… ┆ 22   │
└────────────────┴───────────────────────────────────┴──────┘


In [19]:
with open("geolife-dita-long.txt","w") as f:
    f.writelines([ i[0]+'\n' for i in geolife_data.filter(pl.col("size")>3010).select("points").iter_rows() ])

# Geoflink

In [7]:
import polars as pl
import time
size=44 # 4points/min
with open("geolife-dita-long.txt") as f:
    lines = f.readlines()
start = int(time.time())*1000
long_trajs=[]
for idx,traj in enumerate(lines[:100]):
    id = []
    ts = []
    lon=[]
    lat=[]
    for step,p in enumerate(traj.strip().split(";")):
        id.append(idx)
        ts.append(start+15000*step)
        lon.append(float(p.split(",")[0]))
        lat.append(float(p.split(",")[1]))
    long_trajs.append(pl.DataFrame({"id":id[:size],"ts":ts[:size],"lon":lon[:size],"lat":lat[:size]}))
geoflink_data = pl.concat(
    long_trajs,
    how="vertical",
).sort("ts")
print(geoflink_data)

shape: (4_400, 4)
┌─────┬───────────────┬────────────┬───────────┐
│ id  ┆ ts            ┆ lon        ┆ lat       │
│ --- ┆ ---           ┆ ---        ┆ ---       │
│ i64 ┆ i64           ┆ f64        ┆ f64       │
╞═════╪═══════════════╪════════════╪═══════════╡
│ 0   ┆ 1682841201000 ┆ 116.311859 ┆ 40.00152  │
│ 1   ┆ 1682841201000 ┆ 116.315632 ┆ 40.008224 │
│ 2   ┆ 1682841201000 ┆ 116.322226 ┆ 40.010572 │
│ 3   ┆ 1682841201000 ┆ 116.321513 ┆ 40.007907 │
│ …   ┆ …             ┆ …          ┆ …         │
│ 96  ┆ 1682841846000 ┆ 116.324206 ┆ 40.006219 │
│ 97  ┆ 1682841846000 ┆ 121.52614  ┆ 38.928291 │
│ 98  ┆ 1682841846000 ┆ 121.527505 ┆ 38.92811  │
│ 99  ┆ 1682841846000 ┆ 116.324122 ┆ 40.005258 │
└─────┴───────────────┴────────────┴───────────┘


In [8]:
geoflink_data.write_csv("geolife-geoflink-long.txt",has_header=False)